# Data Pre-Processing 

This code will read in data from the experiments carried out and extract the important information which is fed to the LSTM cell. 

The very first step would be to extract the start and end indices of the data for each subject using the following function:


In [1]:
def start_end_extractor (data, prior_seconds): 
    # data is the dataset; prior_seconds is the number of seconds to be considered in the data
    # before the warning is given 

    warningStartIndices= data[data['warning'].notnull()].index # Extracts the warning indices
    startindex = np.zeros(shape=(len(warningStartIndices),1))
    endindex = np.zeros(shape=(len(warningStartIndices),1))
    ei = 0
    si = 0
    
    for i in warningStartIndices:
        k = 0
        
        # The following WHILE loop identifies the startindex for relevant data
        while ((data['time(s)'][i] - data['time(s)'][i + k]) < (prior_seconds + 0.001) ):
            k -= 1
        j = 0
        
        # The following WHILE loop identifies the endindex for relevant data
        while (abs(data['steering angle(deg)'][i] - data['steering angle(deg)'][i+j]) < 2)\
        & (data['padel1'][i+j] <= 32511):
            j += 1
        
        startindex[si] = i + k 
        si += 1
        endindex[ei] = i + j
        ei += 1
        
        startindex = startindex.astype(int)
        endindex = endindex.astype(int)
    dataSplitVector = endindex - startindex
    
    #print(startindex, endindex, dataSplitVector,warningStartIndices)   
    return startindex, endindex, dataSplitVector,warningStartIndices
    
    

Once we have the relevant start and end points for each event we can extract other information such as the collision/no-collision output for the events as done in the function below:


In [2]:
def output_extractor (data, startindex, endindex):
    # This function takes as its input the unlceaned raw data and the event start and end 
    # indices. It outputs an array which tells whether colision took place for every event
    # for a given subject
    
    output = np.zeros(shape = (endindex.size))
    
    for i in range(endindex.size):
        if i != (endindex.size -1): # The last index will not have a start index for the next event 
            if data['crash'].values[endindex[i]][0] != data['crash'].values[startindex[i+1]]:
            # Comparing the values in the crash column for end index of event i to start index of 
            # event i+1. If these values are different then it suggests there was a collision.
            # This means the output will be 1 = collision
                output[i] = 1
            else:
                output[i] = 0
        else:
            if data['crash'].values[endindex[i]][0] != data['crash'].values[-1]:
                output[i] = 1
            else:
                output[i] = 0
                
    output = pd.DataFrame(output)
            
    return output




The following set of functions will now start modeling the lead vehicle movement in terms of lead vehicle velocity and position:


In [3]:
def event_lenghts (event_sequence, incident_distance, max_distance):
    # This function will take as its input the event_sequence which is taken from the 'event'
    # column of the 'SUMMARY' sheet. The incident_distance and max_distance arguments are the 
    # parameters defined in the experiment setup. The former is the distance at which the 
    # incident takes place and the latter is the max distance over which an event occurs. This
    # function returns the distance from the very start at which every scenario begins and ends.
    

    incident_startpt = np.zeros(shape= (16,1))
    incident_endpt = np.zeros(shape= (16,1))
    
    # dist_counter keeps track of the distance covered
    dist_counter = 0   
    
    for i in range (0,16): 
        # incident_startpt is the distance where the lead vehicle starts decelerating
        incident_startpt[i] = dist_counter + incident_distance[int(event_sequence[i])-1]
        # incident_endpt is the distance where the scenario ends
        incident_endpt[i] = dist_counter + max_distance[int(event_sequence[i])-1]
        dist_counter += max_distance[int(event_sequence[i])-1]
    
    return incident_startpt, incident_endpt

In [4]:
def lead_vehicle_movement(data,event_sequence,scen_data,incident_startpt,incident_endpt,warningStartIndices):
    # This function takes as its input the raw data which has been modified to include the 
    # columns for lead vehicle velocity and distance. It also takes the sequence of events 
    # from the summary sheet, the summary data along with incident_startpt and incident_endpt 
    # as its inputs. 
    
    # dp_counter will keep incrimenting to go to the next data point - dp stands 
    # for data point here
    dp_counter = 0

    # We will check every scenario and match it to the events 1 through 8
    for i in range (0,16):
        # The following condition allows events only from 1,2,3,4,7 and 8 as they have similar
        # lead vehicle movement
        if (int((event_sequence[i]) != 5) & (int(event_sequence[i]) != 6)):
            
            # v_finder keeps track of when the deceleration of lead vehicle starts
            v_finder = 0
            
            # This loop will acertain that all the data points of this event are  
            while (data['dis(feet)'][dp_counter] <= incident_endpt[i]):
                #print(dp_counter)
                # The lead vehicle mimics the movement of the subject vehicle and is always 
                # 100 ft ahead of it before the latter reaches the designated distance for
                # the deceleration to begin. This is already decided in the experiment.
                if data['dis(feet)'][dp_counter] < (incident_startpt[i]-100):
                    data['lvv'][dp_counter] = data['long v(m/s)'][dp_counter]
                    data['lvd'][dp_counter] = data['dis(feet)'][dp_counter] + 100
                    
                # Once the event starts the lead vehicle decelerates to a stop in 2 seconds    
                elif data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                    
                    if v_finder == 0: 
                    # This condition ensures that the following calculations take place only
                    # once per event
                        v_finder += 1
                        init_v = data['long v(m/s)'][dp_counter]       # decc start velocity
                        init_t = data['time(s)'][dp_counter]           # decc start time
                        init_d = data['dis(feet)'][dp_counter] + 100   # decc start position
                        t = 2                                          # time to deccelerate 
                        a = -init_v/t                                  # decceleration

                    # Time elapsed between decceleration start and the current data point
                    T = data['time(s)'][dp_counter]-init_t               
                         
                    # Velocity will keep reducing until it is 0 after which it will stay 0 
                    data['lvv'][dp_counter] = max(0, init_v + a*T)
                    
                    # If the velocity is zero then the distance will stop changing
                    if data['lvv'][dp_counter] == 0:
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1]
                    else:
                        data['lvd'][dp_counter] =  init_d + init_v*T + 0.5*a*T*T

                dp_counter += 1
                if dp_counter == len(data):
                    break

        # The following condition will allow only event 6 with warning lead time of 2.5s
        elif ((int(event_sequence[i]) == 6) & (scen_data['lead time'][i] == 2.5)) :
            
            # v_finder will keep track of the decceleration start
            v_finder = 0
            
            while data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                
                # The lead vehicle will be 50 ft ahead of the subject before the incident 
                # starts and its velocity will be the same as that of the subject vehicle
                if data['dis(feet)'][dp_counter] < (incident_startpt[i] - 50):
                    data['lvv'][dp_counter] = data['long v(m/s)'][dp_counter]
                    data['lvd'][dp_counter] = data['dis(feet)'][dp_counter] + 50
                
                # When the lead vehicle starts to deccelerate it will do so in 1s.
                elif data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                    if v_finder == 0:
                        v_finder += 1
                        init_v = data['long v(m/s)'][dp_counter]     # decc start velocity
                        init_t = data['time(s)'][dp_counter]         # decc start time
                        init_d = data['dis(feet)'][dp_counter] + 50  # decc start position
                        t = 1                                         # time for decceleration
                        a = -init_v/t                                 # decceleration

                    # Time elapsed between decceleration start and the current data point
                    T = data['time(s)'][dp_counter]-init_t               
                         
                    # Velocity will keep reducing until it is 0 after which it will stay 0 
                    data['lvv'][dp_counter] = max(0, init_v + a*T)
                    
                    # If the velocity is zero then the distance will stop changing
                    if data['lvv'][dp_counter] == 0:
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1]
                    else:
                        data['lvd'][dp_counter] =  init_d + init_v*T + 0.5*a*T*T

                dp_counter += 1
                if dp_counter == len(data):
                    break
                
        # The following condition will allow only event 6 with warning lead time of 4.5s
        elif ((int(event_sequence[i]) == 6) & (scen_data['lead time'][i] == 4.5)) :
            
            v_finder = 0
            while data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                
                # The lead vehicle will be 90 ft ahead of the subject before the incident 
                # starts and its velocity will be the same as that of the subject vehicle
                if data['dis(feet)'][dp_counter] < (incident_startpt[i] - 90):
                    data['lvv'][dp_counter] = data['long v(m/s)'][dp_counter]
                    data['lvd'][dp_counter] = data['dis(feet)'][dp_counter] + 90
                
                # When the lead vehicle starts to deccelerate it will do so in 1s.
                elif data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                    if v_finder == 0:
                        v_finder += 1
                        init_v = data['long v(m/s)'][dp_counter]     # decc start velocity
                        init_t = data['time(s)'][dp_counter]         # decc start time
                        init_d = data['dis(feet)'][dp_counter] + 90  # decc start position 
                        t = 1                                        # time for decceleration 
                        a = -init_v/t                                # decceleration

                    # Time elapsed between decceleration start and the current data point
                    T = data['time(s)'][dp_counter]-init_t               
                         
                    # Velocity will keep reducing until it is 0 after which it will stay 0 
                    data['lvv'][dp_counter] = max(0, init_v + a*T)
                    
                    # If the velocity is zero then the distance will stop changing
                    if data['lvv'][dp_counter] == 0:
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1]
                    else:
                        data['lvd'][dp_counter] =  init_d + init_v*T + 0.5*a*T*T

                dp_counter += 1
                if dp_counter == len(data):
                    break

        # The following condition will allow event 5 with warning lead time of 2.5s       
        elif ((int(event_sequence[i]) == 5) & (scen_data['lead time'][i] == 2.5)) :
            
            WT = data['time(s)'][warningStartIndices[i]]            # Waring time instant
            u = data['long v(m/s)'][warningStartIndices[i]] + 66    # Relative velocity of lead vehicle
            import math
            v = -((134.36)**2 + 2*(-66)*179)
            v = math.sqrt(v)                                        # Relative velocity when distance is covered 
            t = (u-v)/66                                            # Time to deccelerate to that velocity
            T = 2.5 - t                                             # Time before WT that decc starts
            decc_start_time = WT + T                                # Time when decc starts
            
            while data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                #print(dp_counter, decc_start_time, WT, u, v , t , T)
                # Before decc_start_time the speed of the oncoming vehicle will be constant at
                # 66 ft/s and its distance will be calculated accordingly
                if data['time(s)'][dp_counter] <= decc_start_time:
                    data['lvv'][dp_counter] = 66
                    data['lvd'][dp_counter] = (decc_start_time - data['time(s)'][dp_counter])\
                    *66 + 179 + data['dis(feet)'][dp_counter]
                    
                elif data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                    data['lvv'][dp_counter] = max(0, (data['lvv'][dp_counter-1] + (data['time(s)'][dp_counter]\
                                                                             - data['time(s)'][dp_counter- 1])*(-66)))
                    if data['lvv'][dp_counter] == 0:
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1]
                    else:
                        time = (data['time(s)'][dp_counter] - data['time(s)'][dp_counter-1])
                        init_v = data['lvv'][dp_counter - 1]
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1] - init_v*time - (0.5*(-66)*(time**2))
                
                dp_counter += 1
                if dp_counter == len(data):
                    break
                
                
        # The following condition will allow event 5 with warning lead time of 4.5s       
        elif ((int(event_sequence[i]) == 5) & (scen_data['lead time'][i] == 4.5)) :
            
            WT = data['time(s)'][warningStartIndices[i]]            # Waring time instant
            u = data['long v(m/s)'][warningStartIndices[i]] + 66    # Relative velocity of lead vehicle
            import math
            v = -((134.36)**2 + 2*(-66)*179)
            v = math.sqrt(v)                                        # Relative velocity when distance is covered 
            t = (u-v)/66                                            # Time to deccelerate to that velocity
            T = 4.5 - t                                             # Time before WT that decc starts
            decc_start_time = WT + T                                # Time when decc starts
            
            while data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                
                # Before decc_start_time the speed of the oncoming vehicle will be constant at
                # 66 ft/s and its distance will be calculated accordingly
                if data['time(s)'][dp_counter] <= decc_start_time:
                    data['lvv'][dp_counter] = 66
                    data['lvd'][dp_counter] = (decc_start_time - data['time(s)'][dp_counter])\
                    *66 + 311 + data['dis(feet)'][dp_counter]
                    
                elif data['dis(feet)'][dp_counter] <= incident_endpt[i]:
                    data['lvv'][dp_counter] = max(0, (data['lvv'][dp_counter-1] + (data['time(s)'][dp_counter]\
                                                                             - data['time(s)'][dp_counter- 1])*(-66)))
                    if data['lvv'][dp_counter] == 0:
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1]
                    else:
                        time = (data['time(s)'][dp_counter] - data['time(s)'][dp_counter-1])
                        init_v = data['lvv'][dp_counter - 1]
                        data['lvd'][dp_counter] = data['lvd'][dp_counter-1] - init_v*time - (0.5*(-66)*(time**2))
                
                dp_counter += 1
                if dp_counter == len(data):
                    break
    
    return data

Now that we have a data frame which also has the lead vehicle movement in it we will work on extracting data relevent to us and store it in a new data frame.

The following function modifies the dataframe to include the columns lvv and lvd and initialise them to 0

In [5]:
def df_modifier_1 (data):
    data['lvv'] = 0
    data['lvd'] = 0
    
    return data

The follwoing function will modify the dataframe to rename the columns and and drop the unnecessary ones

In [6]:
def df_modifier_2 (data, startindex, endindex, dataSplitVector):
    
    # Write the code here
    data2 = np.zeros(shape=(sum(dataSplitVector).astype(int)[0],data.shape[1]))
    j = 0
    for i in range (endindex.size):
        for k in range (startindex[i][0], endindex[i][0]):
            data2[j] = data.values[k]
            j += 1
        print("The no. of time stamps for event {} are {}".format(i+1,(endindex[i][0] - startindex[i][0])))
    featureList = data.columns[0:16]
    #print(featureList)
    data3 = pd.DataFrame(data = data2)
    data3.columns = featureList
    #data3
    
    return data3

The main function is as follows:


In [7]:
def function1(data, scen_data):

    # Step 2: Dropping unnecessary columns
    data = data.drop(data.columns[14:],axis = 1)

    # Step 3: Extracting start and end indices of the data which are relevnt to our model training
    startindex, endindex, dataSplitVector, warningStartIndices = start_end_extractor(data, prior_seconds = 1)

    # Step 4: Extracting the output of the events 
    output = output_extractor(data,startindex,endindex)

    # Step 5: Modifying the initial dataset to add lead vehicle movement columns
    data = df_modifier_1(data)

    # Step 6: Extract the event sequence data from the Summary sheet
    #scen_data = pd.read_excel('/Users/RSM/Downloads/DataToBeCleaned.xlsx', sheet_name = 'SUMMARY')
    #scen_data =scen_data.drop([16,17])  
    event_sequence = scen_data['event'].values

    # Step 7: Extract the incidence start point and end point to model the movement
    incident_distance = [2880,3880,4880,3880,0,1100,2400,880]
    max_distance = [4000,5000,6000,5000,3000,2000,3000,2000]
    incident_startpt, incident_endpt = event_lenghts(event_sequence, incident_distance, max_distance)
    incident_endpt[-1] = data['dis(feet)'][len(data['dis(feet)'])-1]

    # Step 8: Model the lead vehicle movement and update the 'lvv' and 'lvd' columns
    data2 = lead_vehicle_movement(data,event_sequence,scen_data,incident_startpt,incident_endpt, warningStartIndices)

    # Step 9: Extract the relevant portion of the data 
    data_rel = df_modifier_2(data2, startindex, endindex, dataSplitVector)
    
    return data_rel, dataSplitVector, output

In [8]:
import pandas as pd
import numpy as np

filepaths = pd.read_excel('/Users/RSM/Desktop/cvs_data/filepaths.xlsx', header = None)

input_data = pd.DataFrame()
output_data = pd.DataFrame()
splitVector = pd.DataFrame()
    

for i in range (len(filepaths)):
    filepath = filepaths[0][i]
    data = pd.read_excel(filepath, sheet_name = 'Data')
    scen_data = pd.read_excel(filepath, sheet_name = 'SUMMARY')
    cleaned_data, dataSplitVector, output = function1(data, scen_data)
    dataSplitVector = pd.DataFrame(dataSplitVector)
    input_data = input_data.append(cleaned_data, ignore_index = True)
    output_data = output_data.append(output, ignore_index = True)
    splitVector = splitVector.append(dataSplitVector, ignore_index = True)
    print("Data for Subject {} has been cleaned.".format(i+1))


/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a 

The no. of time stamps for event 1 are 19
The no. of time stamps for event 2 are 18
The no. of time stamps for event 3 are 14
The no. of time stamps for event 4 are 19
The no. of time stamps for event 5 are 31
The no. of time stamps for event 6 are 31
The no. of time stamps for event 7 are 16
The no. of time stamps for event 8 are 50
The no. of time stamps for event 9 are 19
The no. of time stamps for event 10 are 17
The no. of time stamps for event 11 are 16
The no. of time stamps for event 12 are 17
The no. of time stamps for event 13 are 15
The no. of time stamps for event 14 are 16
The no. of time stamps for event 15 are 17
The no. of time stamps for event 16 are 15
Data for Subject 1 has been cleaned.


/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:154: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:156: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:166: SettingWithCopyWarning: 
A value is trying to be set o

The no. of time stamps for event 1 are 22
The no. of time stamps for event 2 are 18
The no. of time stamps for event 3 are 16
The no. of time stamps for event 4 are 19
The no. of time stamps for event 5 are 18
The no. of time stamps for event 6 are 21
The no. of time stamps for event 7 are 17
The no. of time stamps for event 8 are 16
The no. of time stamps for event 9 are 16
The no. of time stamps for event 10 are 16
The no. of time stamps for event 11 are 11
The no. of time stamps for event 12 are 13
The no. of time stamps for event 13 are 16
The no. of time stamps for event 14 are 15
The no. of time stamps for event 15 are 16
The no. of time stamps for event 16 are 16
Data for Subject 2 has been cleaned.
The no. of time stamps for event 1 are 17
The no. of time stamps for event 2 are 17
The no. of time stamps for event 3 are 14
The no. of time stamps for event 4 are 17
The no. of time stamps for event 5 are 15
The no. of time stamps for event 6 are 16
The no. of time stamps for event

The no. of time stamps for event 1 are 22
The no. of time stamps for event 2 are 14
The no. of time stamps for event 3 are 16
The no. of time stamps for event 4 are 20
The no. of time stamps for event 5 are 15
The no. of time stamps for event 6 are 15
The no. of time stamps for event 7 are 18
The no. of time stamps for event 8 are 16
The no. of time stamps for event 9 are 17
The no. of time stamps for event 10 are 15
The no. of time stamps for event 11 are 15
The no. of time stamps for event 12 are 15
The no. of time stamps for event 13 are 16
The no. of time stamps for event 14 are 25
The no. of time stamps for event 15 are 19
The no. of time stamps for event 16 are 23
Data for Subject 14 has been cleaned.
The no. of time stamps for event 1 are 21
The no. of time stamps for event 2 are 18
The no. of time stamps for event 3 are 18
The no. of time stamps for event 4 are 20
The no. of time stamps for event 5 are 65
The no. of time stamps for event 6 are 18
The no. of time stamps for even

/Users/RSM/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


The no. of time stamps for event 1 are 23
The no. of time stamps for event 2 are 19
The no. of time stamps for event 3 are 17
The no. of time stamps for event 4 are 10
The no. of time stamps for event 5 are 16
The no. of time stamps for event 6 are 10
The no. of time stamps for event 7 are 31
The no. of time stamps for event 8 are 32
The no. of time stamps for event 9 are 16
The no. of time stamps for event 10 are 15
The no. of time stamps for event 11 are 14
The no. of time stamps for event 12 are 9
The no. of time stamps for event 13 are 14
The no. of time stamps for event 14 are 13
The no. of time stamps for event 15 are 12
The no. of time stamps for event 16 are 14
Data for Subject 17 has been cleaned.
The no. of time stamps for event 1 are 34
The no. of time stamps for event 2 are 22
The no. of time stamps for event 3 are 25
The no. of time stamps for event 4 are 22
The no. of time stamps for event 5 are 19
The no. of time stamps for event 6 are 27
The no. of time stamps for event

The no. of time stamps for event 9 are 16
The no. of time stamps for event 10 are 18
The no. of time stamps for event 11 are 17
The no. of time stamps for event 12 are 14
The no. of time stamps for event 13 are 18
The no. of time stamps for event 14 are 19
The no. of time stamps for event 15 are 25
The no. of time stamps for event 16 are 17
Data for Subject 28 has been cleaned.
The no. of time stamps for event 1 are 52
The no. of time stamps for event 2 are 40
The no. of time stamps for event 3 are 22
The no. of time stamps for event 4 are 21
The no. of time stamps for event 5 are 17
The no. of time stamps for event 6 are 26
The no. of time stamps for event 7 are 22
The no. of time stamps for event 8 are 25
The no. of time stamps for event 9 are 18
The no. of time stamps for event 10 are 25
The no. of time stamps for event 11 are 18
The no. of time stamps for event 12 are 10
The no. of time stamps for event 13 are 21
The no. of time stamps for event 14 are 16
The no. of time stamps for

In [10]:
input_data = input_data.drop(['crash', 'trans_gear', 'trans gear', 'warning'], axis = 1)

In [11]:
#output_copy = output_data[:]
#output_copy = output_copy.drop(splitVector[splitVector[0] == 0].index)

In [12]:
output_data.size

512

In [13]:
#output_copy.size

In [14]:
splitVector[splitVector[0]==0].index

Int64Index([480], dtype='int64')

In [ ]:
416/16

In [15]:
input_data.to_excel('cleaned_data_version_01.xlsx')
output_data.to_excel('output_data_version_01.xlsx')
splitVector.to_excel('data_split_version_01.xlsx')

# Modeling the LSTM

This part of the code will do some further preprocessing of the data to make it ready for the LSTM. Follwoing which, an LSTM model will be built, trained and tested. 

In [1]:
import torch 
import torch.nn as nn
from torch.autograd import Variable

# Defining the LSTM model as a class. The model we will train later will be the instance of this class.

class collisionClassifier(nn.Module):
    def __init__(self, no_of_features, features_to_hidden, features_in_hidden, max_sequence_length, no_of_layers = 1):
        super().__init__()
        self.features_in_hidden = features_in_hidden
        self.features_to_hidden = features_to_hidden
        self.no_of_layers = no_of_layers
        self.max_sequence_length = max_sequence_length
        self.linear_layer_1 = nn.Linear(no_of_features, features_to_hidden)
        self.lstm_cell = nn.LSTM(features_to_hidden, features_in_hidden, no_of_layers)
        self.dropout_layer = nn.Dropout(0.2)
        self.relu_layer = nn.ReLU()
        self.linear_layer_2 = nn.Linear(features_in_hidden,2)
        self.smc = nn.Softmax()
        
    def __init__hidden(self, max_sequence_length):
        hidden = torch.rand(1, max_sequence_length, self.features_in_hidden)
        return Variable(hidden, requires_grad = True)
        
    def __init__cellstate(self, max_sequence_length):
        cellstate = torch.rand(1,max_sequence_length , self.features_in_hidden)
        return Variable(cellstate, requires_grad = True)
    
    def forward_pass(self, input_data,sequence_length):
        output = self.linear_layer_1(input_data)
        output, _ = self.lstm_cell(output)
        output = self.relu_layer(output[0,int(sequence_length-1),:])
        output = self.linear_layer_2(output)
        output = self.smc(output)     #hidden[:,(sequence_length-1),:])
        return output

In [2]:
def data_multiplier_and_shaper(driving_data, output_data, data_split_vector, multiplier):
    driving_data_large = driving_data
    output_data_large = output_data
    data_split_vector_large = data_split_vector
    for i in range (0,multiplier):
        driving_data_large = np.append(driving_data_large, driving_data)
        output_data_large = np.append(output_data_large,output_data)
        data_split_vector_large = np.append(data_split_vector_large, data_split_vector)
    x = 1*driving_data.shape[0]
    driving_data_large = driving_data_large.reshape(x,13)
    a = len(data_split_vector)
    b = len(data_split_vector_large)
    for i in range(a,b):
        driving_data_large[a,:] = driving_data_large[a,:] + \
        np.random.normal(loc = 0, scale = 1, size = (1,driving_data.shape[1]))

    return data_scaler(driving_data_large, data_split_vector_large) , output_data_large, data_split_vector_large

In [3]:
def data_scaler(driving_data, data_split_vector):
    from sklearn.preprocessing import MinMaxScaler

    no_of_data_points = driving_data.shape[0]
    no_of_columns = driving_data.shape[1]

    scaler = MinMaxScaler(feature_range = (0,1))
    scaledData = scaler.fit_transform(driving_data)
    scaledData = scaledData.reshape([1, no_of_data_points, no_of_columns])
    
    return paddingMethod(scaledData, data_split_vector)


In [4]:
def paddingMethod(scenario_data, data_split_vector):
    scenario_tensor = torch.zeros((len(data_split_vector),max(max(data_split_vector)),scenario_data.shape[2])).float()

    j = 0    
    for idx, scen_length in enumerate(data_split_vector):
        for i in range (max(scen_length)):
            scenario_tensor[idx,i,:] = torch.FloatTensor(scenario_data[0,j+i,:])
        j += max(scen_length)
        
    print("The input data shape is {}".format(scenario_tensor.shape))
    
    return scenario_tensor

In [5]:
def avg_error(linear_output, target):
    a = [1,1]
    a = torch.Tensor(a)
    loss = nn.CrossEntropyLoss(weight = a)
    error = loss(linear_output,target)
    return error


In [6]:
def test_train_split(k_folds, input_data, output_data, data_split_vector, k): # k is the fold number
    #k_folds = 8
    independent_events = input_data.size()[0]
    data_per_fold = int(independent_events/k_folds)
    train_events = (k_folds - 1)*data_per_fold    #int(independent_events*0.8)
    input_test_data = torch.zeros((data_per_fold,input_data.size()[1],input_data.size()[2]))
    input_train_data = torch.zeros((data_per_fold*(k_folds-1),input_data.size()[1],input_data.size()[2]))
    input_train_target = torch.zeros((data_per_fold*(k_folds - 1)))
    input_train_target = input_train_target.type(torch.LongTensor)
    train_datasplit = np.zeros([1,data_per_fold*(k_folds - 1)])
    test_datasplit = np.zeros([1, data_per_fold])
    
    counter = 0 
    for l in range (k_folds):
        if l == k:
            input_test_data = input_data[l*data_per_fold:(l+1)*data_per_fold,:,:]
            test_target = output_data[l*data_per_fold:(l+1)*data_per_fold,0]
            test_datasplit = data_split_vector[l*data_per_fold:(l+1)*data_per_fold,0]
        else:
            input_train_data[counter*data_per_fold:(counter+1)*data_per_fold,:,:] = input_data[l*data_per_fold:\
                                                                                               (l+1)*data_per_fold,:,:]
            input_train_target[counter*data_per_fold:(counter+1)*data_per_fold]= output_data[l*data_per_fold:(l+1)*data_per_fold,0] 
            train_datasplit[0,counter*data_per_fold:(counter+1)*data_per_fold] = data_split_vector[l*data_per_fold:(l+1)*data_per_fold,0]
            counter += 1
            
    return input_train_data, input_train_target, input_test_data, test_target, train_events, train_datasplit, test_datasplit

In [7]:
def model_init(input_train_data, input_train_target, no_of_batches, data_split_vector, train_datasplit, epoch):
    
    collision_predictor = collisionClassifier(no_of_features = input_train_data.size()[2],features_to_hidden = 10, \
                                          features_in_hidden = 2, max_sequence_length = max(data_split_vector))
    
    batch_size = int(input_train_data.size()[0]/no_of_batches)
    optimizer = torch.optim.Adam(collision_predictor.parameters(), lr=0.001)
    
    return model_trainer(input_train_data, input_train_target, no_of_batches, data_split_vector,\
                         train_datasplit, batch_size, optimizer, collision_predictor, epoch)

In [8]:
def model_trainer(input_train_data, input_train_target, no_of_batches, data_split_vector,\
                         train_datasplit, batch_size, optimizer, collision_predictor, epoch):
    
    for i in range(epoch):
        for j in range(no_of_batches):
            pred_tensor = torch.zeros(batch_size,2)
            target = torch.LongTensor(input_train_target)
            target_tensor = target[j*batch_size:(j+1)*batch_size] 
            for k in range (batch_size):
                event_no = (batch_size*j)+k
                prediction = collision_predictor.forward_pass(input_train_data[event_no,:,:].unsqueeze(0),\
                                                                      train_datasplit[0,event_no])
                pred_tensor[k] = prediction
        
            error = avg_error(pred_tensor,target_tensor)
            collision_predictor.zero_grad()
            error.backward()
            optimizer.step()
        
        print("The error for epoch {} was: {}".format((i+1),error))
            
    return collision_predictor

In [9]:
def test(input_test_data, test_target, collision_predictor, test_datasplit):
    
    test_target = np.array(test_target)
    test_predictions = np.zeros(test_target.shape)
    event_no = 0
    for i in range(0,input_test_data.shape[0]):
        test_output = collision_predictor.forward_pass(input_test_data[i,:,:].unsqueeze(0), \
                                                                test_datasplit[event_no])
        if test_output[0] < test_output[1]:
            test_predictions[i] = 1
        else :
            test_predictions[i] = 0
        event_no += 1
    tp = 0
    fp = 0
    fn = 0
    tn = 0
    for i in range(0, input_test_data.shape[0]):
        if ((test_predictions[i] == test_target[i]) & (test_predictions[i] == 1)):
            tp += 1
        if ((test_predictions[i] == test_target[i]) & (test_predictions[i] == 0)):
            tn += 1
        elif((test_predictions[i] != test_target[i]) & (test_predictions[i] == 1)):
            fp += 1
        elif((test_predictions[i] != test_target[i]) & (test_predictions[i] == 0)):
            fn += 1
    return tp, fp, fn, tn, test_predictions

In [10]:
def main_function(driving_data, output_data,data_split_vector,multiplier = 0, k_folds = 8,no_of_batches = 4, epoch = 20):

    input_data, output, data_split_vector = data_multiplier_and_shaper(driving_data, output_data,data_split_vector,multiplier)
    print("There are {} inputs to the LSTM. How many folds do you want for your cross-validation, default being 8?".format(input_data.shape[0]))
    k_folds = input()
    k_folds = int(k_folds)
    print("What is the number of batches you desire the training data be split into? Default is set at 4")
    no_of_batches = input()
    no_of_batches = int(no_of_batches)
    print("What is the number of epochs for which the training should be carried out? Default is set at 20")
    epoch = input()
    epoch = int(epoch)
    output_data = torch.LongTensor(output_data)
    true_positive  = 0
    false_positive = 0
    false_negative = 0
    true_negative = 0
    for k in range(k_folds):
        input_train_data, input_train_target, input_test_data, test_target,train_events, \
        train_datasplit, test_datasplit = test_train_split(k_folds, input_data, output_data, data_split_vector, k)
        collision_predictor = model_init(input_train_data, input_train_target, no_of_batches, data_split_vector, train_datasplit, epoch)
        tp, fp, fn, tn, test_predictions = test(input_test_data, test_target, collision_predictor, test_datasplit)
        true_positive  += tp
        false_positive += fp
        false_negative += fn
        true_negative  += tn

    #confusion_matrix = pd.DataFrame(np.array([true_positive,true_negative],[false_positive, false_negative]),columns = ['Positive (Collision)','Negative (No Collision)'], index = ['true','false'])
    
    return true_positive,true_negative,false_positive, false_negative, test_predictions
                     

In [11]:
import numpy as np
import pandas as pd

# importing the time series input data

filepath_main_data = "/Users/RSM/cleaned_data_version_01.xlsx"
driving_data = pd.read_excel(filepath_main_data)
driving_data = driving_data.values

# importing the collision outcome for the corresponding input data

filepath_output = "/Users/RSM/output_data_version_01.xlsx"
output_data = pd.read_excel(filepath_output)
output_data = output_data.values

# importing the data split vector which stores the length of each time series data

filepath_data_split = "/Users/RSM/data_split_version_01.xlsx"
data_split_vector = pd.read_excel(filepath_data_split)
data_split_vector = data_split_vector.values

print("The shape of the input driving data is:{}\
The shape of the output data is: {}\
The shape of the data split vector is: {}".format(driving_data.shape, output_data.shape, data_split_vector.shape))


The shape of the input driving data is:(10258, 13)The shape of the output data is: (512, 1)The shape of the data split vector is: (512, 1)


In [25]:

true_positive,true_negative,false_positive, false_negative, test_predictions = main_function(driving_data, output_data,data_split_vector,multiplier = 0, k_folds = 8,no_of_batches = 4, epoch = 20)


The input data shape is torch.Size([512, 67, 13])
There are 512 inputs to the LSTM. How many folds do you want for your cross-validation, default being 8?
8
What is the number of batches you desire the training data be split into? Default is set at 4
4
What is the number of epochs for which the training should be carried out? Default is set at 20
3


/Users/RSM/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


The error for epoch 1 was: 0.5995200872421265
The error for epoch 2 was: 0.5989611744880676
The error for epoch 3 was: 0.5984047055244446
The error for epoch 1 was: 0.7172980904579163
The error for epoch 2 was: 0.7161141633987427
The error for epoch 3 was: 0.7149358987808228
The error for epoch 1 was: 0.620996356010437
The error for epoch 2 was: 0.6202690005302429
The error for epoch 3 was: 0.6195478439331055
The error for epoch 1 was: 0.5895795226097107
The error for epoch 2 was: 0.5890313386917114
The error for epoch 3 was: 0.5884881019592285
The error for epoch 1 was: 0.7757959365844727
The error for epoch 2 was: 0.7739117741584778
The error for epoch 3 was: 0.7721189260482788
The error for epoch 1 was: 0.6337165832519531
The error for epoch 2 was: 0.632579505443573
The error for epoch 3 was: 0.6314259171485901
The error for epoch 1 was: 0.6439838409423828
The error for epoch 2 was: 0.6410676836967468
The error for epoch 3 was: 0.6385313868522644
The error for epoch 1 was: 0.6472273

In [26]:
print("true positive: {} \
true negative: {} \
false positive: {} \
false negative: {}".format(true_positive,true_negative,false_positive, false_negative))

true positive: 30 true negative: 271 false positive: 98 false negative: 113


In [27]:
print("accuracy: {}".format((true_positive + true_negative)/(true_positive + true_negative+ false_positive + false_negative)))

accuracy: 0.587890625


In [19]:
# warrning parameters, scenario (cross-section and stratight), warning reliability, timing and type 
# gaussian error

224